In [8]:
!pip install optuna

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import optuna
import numpy as np
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the Reuters dataset
max_len = 300  # Adjust based on the dataset analysis
(X_train, y_train), (X_test, y_test) = reuters.load_data(path="reuters.npz")
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create a DataLoader
batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, n_hidden, n_units, dropout_rate):
        super(NeuralNetwork, self).__init__()
        layers = [nn.Linear(input_dim, n_units), nn.ReLU(), nn.Dropout(dropout_rate)]

        for _ in range(n_hidden):
            layers += [nn.Linear(n_units, n_units), nn.ReLU(), nn.Dropout(dropout_rate)]

        layers += [nn.Linear(n_units, output_dim)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        logits = self.network(x)
        return logits

def create_model(trial, input_dim, output_dim):
    n_hidden = trial.suggest_int('n_hidden', 3, 7)  # Increased range for hidden layers
    n_units = trial.suggest_int('n_units', 64, 256)  # Increased range for units per layer
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)  # Suggesting dropout rate
    model = NeuralNetwork(input_dim, output_dim, n_hidden, n_units, dropout_rate)
    return model


def objective(trial):
    model = create_model(trial, X_train.shape[1], len(np.unique(y_train)))
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(10):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            _, predicted = torch.max(output.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    return accuracy



In [10]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10, n_jobs=-1)  # Reduced the number of trials for brevity

print(study.best_params)

# Create a model with the best hyperparameters found
best_model = create_model(study.best_trial, X_train.shape[1], len(np.unique(y_train)))

[I 2024-04-29 23:28:19,913] A new study created in memory with name: no-name-f2ac02da-005b-4db4-b970-8af254585a29
<ipython-input-9-6ebd1db886f0>:56: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
[I 2024-04-29 23:28:46,542] Trial 0 finished with value: 0.41585040071237755 and parameters: {'n_hidden': 4, 'n_units': 118, 'dropout_rate': 0.1635270194911682, 'learning_rate': 0.00010649115779302927}. Best is trial 0 with value: 0.41585040071237755.
[I 2024-04-29 23:29:03,082] Trial 1 finished with value: 0.37310774710596617 and parameters: {'n_hidden': 6, 'n_units': 231, 'dropout_rate': 0.1662249356461894, 'learning_rate': 0.00014561113817368877}. Best is trial 0 with value: 0.41585040071237755.
[I 2024-04-29 23:29:18,402] Trial 2 finished with value: 0.37043633125

{'n_hidden': 4, 'n_units': 118, 'dropout_rate': 0.1635270194911682, 'learning_rate': 0.00010649115779302927}


In [11]:
# Evaluate the best model on test data
best_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = best_model(X_batch)
        _, predicted = torch.max(output.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

accuracy = correct / total
print(f"Accuracy of the best model: {accuracy}")

Accuracy of the best model: 0.012911843276936777
